In [1]:
import os
import json
from IPython.display import Image, display

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.util.tools_fabric import ToolsFabric

from react_agent.src.util.llm_proxy import LLM_PROXY

from experiments.util.fact_score.fact_scorer import FactScorer

In [2]:
USE_MCP = False
fact_scorer = FactScorer()
path_to_ressources = "./ressources/"

files = {
    "extensibility": "extensibility_assistance_result.json",
    "malaysia": "malaysia_support_result.json",
    "peppol": "peppol_support_result.json",
}

In [3]:
file_path = os.path.abspath(path_to_ressources + files["extensibility"])


with open(file_path, encoding="utf8") as f:
    json_file = json.load(f)

In [4]:
case = json_file["cases"][2]
print("Case ID: ", case["id"])

Case ID:  EA-3


In [5]:
triage_agent = Triage()

query_categorization = triage_agent.triage_user_message(
    user_message=case["question"]
)
print(query_categorization)

{'user_query': 'For a Greece customer invoice process in Public Cloud, can I enhance or extend the generated XML before it is sent to the authorities?', 'category': 'Knowledge-QA'}


In [6]:
tools = ToolsFabric.get_tools_for_category( use_mcp = USE_MCP, configuration = query_categorization["category"] )

In [7]:
if USE_MCP:
    async with MultiServerMCPClient( tools ) as client:
        agent = ReActAgent(tool_list=client.get_tools())
        await agent.arun_and_print_agent_stream(
            user_message=query_categorization["user_query"]
        )
else:
    agent = ReActAgent(tool_list=tools)
    agent.run_and_print_agent_stream(user_message=query_categorization["user_query"])

================================ Human Message =================================

For a Greece customer invoice process in Public Cloud, can I enhance or extend the generated XML before it is sent to the authorities?
================================== Ai Message ==================================

Observation:
The user is asking whether it's possible to enhance or extend the XML generated for a Greece customer invoice process in SAP Public Cloud before it is sent to the authorities.

Thought:
To answer this question, I need to determine if SAP Document and Reporting Compliance allows for customization or extension of the XML output for Greece customer invoices. This involves understanding the capabilities and limitations of the system in terms of XML manipulation and compliance requirements.

Action Plan:
1. Retrieve relevant information about customizing or extending XML in SAP Document and Reporting Compliance for Greece invoices.
2. Check if there are specific guidelines or restrict

In [8]:
agent_answer = agent.response_message.content

In [9]:
output = await fact_scorer.classify_facts_in_context(facts=case.get("facts"), knowledge_source=agent_answer)

for result in output:
    print(f"Fact: {result.fact}")
    print(f"Contained: {result.is_contained}")
    print(f"Reason: {result.reason}")
    print("-" * 40)

Fact: You can enhance the generated XML before it is sent to the authorities.
Contained: True
Reason: The context explicitly states that you can enhance or extend the generated XML for Greece customer invoices in SAP Public Cloud before it is sent to the authorities.
----------------------------------------
Fact: You can extend the generated XML before it is sent to the authorities.
Contained: True
Reason: The context explicitly states that you can enhance or extend the generated XML for Greece customer invoices in SAP Public Cloud before it is sent to the authorities.
----------------------------------------
Fact: The enhancement or extension is done using the method SET_OUTPUT_DATA.
Contained: False
Reason: The context mentions various tools and methods for enhancing or extending XML output, but does not specifically mention the method SET_OUTPUT_DATA.
----------------------------------------
Fact: The method SET_OUTPUT_DATA is part of BAdI EDOC_ADAPTOR_CLOUD.
Contained: False
Reason

In [10]:
LLM_PROXY.print_usage()

Call Count: 8
Token Usage: {'input_tokens': 16910, 'output_tokens': 1832, 'total_tokens': 18742}
